In [1]:
# !pip install darts

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.8 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 23.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 25.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MinMaxScaler
from darts.metrics import rmse, mae
import matplotlib.pyplot as plt
import random
from pytorch_lightning.callbacks import EarlyStopping
import torch

In [3]:
# 1. خواندن داده
df = pd.read_excel("main_trans.xlsx")
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df = df.dropna(subset=['DateTime'])

In [4]:
# 2. فیلتر بازه زمانی
target_date = pd.to_datetime('2023-9-01 00:00:00')
df_filtered = df[df['DateTime'] >= target_date]

In [5]:
# 3. بازنمونه‌گیری و پر کردن داده‌های خالی
df_filtered = df_filtered.set_index('DateTime').resample('1H').mean()
df_filtered = df_filtered.interpolate(method='linear')

/tmp/ipykernel_936/4011370313.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_filtered = df_filtered.set_index('DateTime').resample('1H').mean()


In [6]:
# 4. ساخت سری هدف و covariates
series = TimeSeries.from_dataframe(df_filtered, value_cols='oil temperature', freq='h')
covariates = TimeSeries.from_dataframe(df_filtered, value_cols=['winding temperature', 'tap position'], freq='h')

In [7]:
# 5. نرمال‌سازی
scaler_series = Scaler(MinMaxScaler())
series_scaled = scaler_series.fit_transform(series)

scaler_cov = Scaler(MinMaxScaler())
covariates_scaled = scaler_cov.fit_transform(covariates)

In [8]:
# 6. تقسیم داده‌ها به train, val, test
train_ratio = 0.8
val_ratio = 0.1
train_series, temp_series = series_scaled.split_after(train_ratio)
val_series, test_series = temp_series.split_after(val_ratio / (1 - train_ratio))
train_cov, temp_cov = covariates_scaled.split_after(train_ratio)
val_cov, test_cov = temp_cov.split_after(val_ratio / (1 - train_ratio))

In [9]:
# تعریف فضای پارامترها برای random search
param_space = {
    'input_chunk_length': [24, 48, 72],
    'output_chunk_length': [6, 12, 24],   
    'num_layers': [2, 4, 6],
    'layer_widths': [256, 512, 1024]
}

In [13]:
# تعداد آزمون‌های random search
n_trials =10

best_model = None
best_params = None
best_val_rmse = float('inf')

In [14]:
# 7. Random Search برای تیونینگ
for _ in range(n_trials):
    # انتخاب random پارامترها
    params = {k: random.choice(v) for k, v in param_space.items()}
    print(f"Testing params: {params}")
    
    # ساخت مدل با early stopping
    pl_trainer_kwargs = {
        "callbacks": [EarlyStopping(monitor="val_loss", patience=5, mode="min")],
        "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
        "devices": 1
    }
    
    model = NBEATSModel(
        input_chunk_length=params['input_chunk_length'],
        output_chunk_length=params['output_chunk_length'],
        n_epochs=100,  # حداکثر epochs، early stopping متوقف می‌کنه
        num_stacks=30,
        num_blocks=3,
        num_layers=params['num_layers'],
        layer_widths=params['layer_widths'],
        batch_size=32,
        random_state=42,
        model_name="oil-temp-nbeats",
        pl_trainer_kwargs=pl_trainer_kwargs
    )

Testing params: {'input_chunk_length': 72, 'output_chunk_length': 6, 'num_layers': 4, 'layer_widths': 512}
Testing params: {'input_chunk_length': 48, 'output_chunk_length': 12, 'num_layers': 2, 'layer_widths': 512}
Testing params: {'input_chunk_length': 24, 'output_chunk_length': 6, 'num_layers': 4, 'layer_widths': 256}
Testing params: {'input_chunk_length': 72, 'output_chunk_length': 6, 'num_layers': 4, 'layer_widths': 256}
Testing params: {'input_chunk_length': 24, 'output_chunk_length': 6, 'num_layers': 2, 'layer_widths': 256}
Testing params: {'input_chunk_length': 72, 'output_chunk_length': 24, 'num_layers': 2, 'layer_widths': 256}
Testing params: {'input_chunk_length': 24, 'output_chunk_length': 6, 'num_layers': 2, 'layer_widths': 512}
Testing params: {'input_chunk_length': 72, 'output_chunk_length': 12, 'num_layers': 2, 'layer_widths': 256}
Testing params: {'input_chunk_length': 48, 'output_chunk_length': 6, 'num_layers': 2, 'layer_widths': 512}
Testing params: {'input_chunk_leng

In [15]:
# 8. آموزش مدل با validation برای early stopping
model.fit(
        series=train_series,
        past_covariates=train_cov,
        val_series=val_series,
        val_past_covariates=val_cov,
        verbose=True
    )
    
# پیش‌بینی روی validation برای ارزیابی
pred_val = model.predict(n=len(val_series), series=train_series, past_covariates=train_cov[-params['input_chunk_length']:].append(val_cov))
pred_val = scaler_series.inverse_transform(pred_val)
val_series_orig = scaler_series.inverse_transform(val_series)
    
val_rmse = rmse(val_series_orig, pred_val)
print(f"Validation RMSE: {val_rmse:.2f}")
    
# ذخیره بهترین مدل
if val_rmse < best_val_rmse:
    best_val_rmse = val_rmse
    best_model = model
    best_params = params

print(f"Best params: {best_params}")
print(f"Best validation RMSE: {best_val_rmse:.2f}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 493 M  | train
-------------------------------------------------------------
493 M     Trainable params
6.4 K     Non-trainable params
493 M     Total params
1,973.470 Total estimated model params size (MB)
1236      Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Validation RMSE: 2.59
Best params: {'input_chunk_length': 72, 'output_chunk_length': 6, 'num_layers': 6, 'layer_widths': 1024}
Best validation RMSE: 2.59


In [17]:
# 9. پیش‌بینی روی test با بهترین مدل
# pred = best_model.predict(n=len(test_series), series=train_series.append(val_series), past_covariates=covariates_scaled[:len(train_series) + len(val_series)])
# pred = scaler_series.inverse_transform(pred)
# test_series = scaler_series.inverse_transform(test_series)

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
ValueError: For the given forecasting horizon `n=72`, the provided `past_covariates` at series sequence index `0` do not extend far enough into the future. As `n > output_chunk_length` the `past_covariates` must end at or after time step `2023-11-26 11:00:00`, whereas now the end is at time step `2023-11-23 17:00:00`.


ValueError: For the given forecasting horizon `n=72`, the provided `past_covariates` at series sequence index `0` do not extend far enough into the future. As `n > output_chunk_length` the `past_covariates` must end at or after time step `2023-11-26 11:00:00`, whereas now the end is at time step `2023-11-23 17:00:00`.

In [18]:
# محاسبه طول مورد نیاز برای covariates
required_covariates_length = len(train_series) + len(val_series) + len(test_series)

# بررسی اینکه آیا covariates کافی هستند
if len(covariates_scaled) < required_covariates_length:
    raise ValueError(f"طول covariates کافی نیست. مورد نیاز: {required_covariates_length}، موجود: {len(covariates_scaled)}")

# اجرای پیش‌بینی با covariates کامل
pred = best_model.predict(
    n=len(test_series),
    series=train_series.append(val_series),
    past_covariates=covariates_scaled[:required_covariates_length]
)

# تبدیل مقیاس خروجی‌ها
pred = scaler_series.inverse_transform(pred)
test_series = scaler_series.inverse_transform(test_series)


`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [19]:
# 10. ارزیابی روی test
print(f"Test RMSE: {rmse(test_series, pred):.2f}")
print(f"Test MAE: {mae(test_series, pred):.2f}")

Test RMSE: 3.53
Test MAE: 3.05


In [20]:
# 11. تبدیل به DataFrame برای رسم
df_test = pd.DataFrame({'oil temperature': test_series.values().flatten()}, index=test_series.time_index)
df_pred = pd.DataFrame({'oil temperature': pred.values().flatten()}, index=pred.time_index)

In [ ]:
# 12. رسم فقط یک روز از داده‌های تست و پیش‌بینی
start_time = df_test.index[0].replace(hour=0, minute=0, second=0)
end_time = start_time + pd.Timedelta(hours=23)

df_test_day = df_test[(df_test.index >= start_time) & (df_test.index <= end_time)]
df_pred_day = df_pred[(df_pred.index >= start_time) & (df_pred.index <= end_time)]

plt.figure(figsize=(6, 3))
plt.plot(df_test_day.index, df_test_day['oil temperature'], label='Actual', color='blue')
plt.plot(df_pred_day.index, df_pred_day['oil temperature'], label='Predicted', color='orange')
plt.legend()
plt.title(f"Actual vs Predicted Oil Temperature\n{start_time.date()} 00:00 to 23:00")
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.grid(True)
plt.tight_layout()
plt.show()